In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import jieba
from jieba import analyse
from PIL import Image
from collections import Counter
import plotly.express as px

In [2]:
data_text = []
with open('chat.txt') as f:
    while True:
        line = f.readline()
        if not line:
            break
        try:
            i = line.index('http')
        except ValueError: 
            if line[:2] in '下午' or line[:2] in '上午':
                line = line[ line.index('\t', 8)+1 :]
                data_text.append(line.replace('\n', ''))

In [3]:
data_processing1 = ''
for sentence in data_text:
    data_processing1 = data_processing1 + sentence

### Keywords -> Word2Vec
#### TextProcessing--stopwords(1)

1.分詞

In [4]:
jieba.load_userdict ('userdict.txt')
data_word1 = jieba.lcut( data_processing1 )

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/n5/l98y_t4948n_shwq2cwgyrgr0000gn/T/jieba.cache
Loading model cost 0.564 seconds.
Prefix dict has been built succesfully.


2.除去停用詞

In [5]:
read_file = pd.read_csv( 'stopword1.txt' )
stopword1 = []
for i in read_file:
    string = read_file[i] 
    stopword1.extend( string )
while i!=0:
    i=0
    for string in data_word1:
        if string in stopword1:
            data_word1.remove(string)
            i += 1

In [6]:
word = Counter(data_word1)
words_most = Counter( word ).most_common(5)
stopword_custom=[]
for i in range(5):
    stopword_custom.append( words_most[i][0] )
while i!=0:
    i=0
    for string in data_word1:
        if string in stopword_custom:
            data_word1.remove(string)
            i += 1

3.調整分詞形式

In [7]:
for i in range(len(data_word1)):
    if( data_word1[i]=='24'): data_word1[i]='924'
    elif( data_word1[i]=='25'): data_word1[i]='925'
    elif( data_word1[i]=='9'): data_word1[i]=''
    elif( data_word1[i]=='10'): data_word1[i]='101'
    elif( data_word1[i]=='1'): data_word1[i]=''

4.除去中文注音讀音、英文（期望Word2Vec組成單純的中文句子）

In [8]:
cn = ['˙','ˊ','ˇ','ˋ']
token1=[]
for string in data_word1:
    if string.isalpha() and not(string.islower()) and (string not in cn) or string.isdigit() :
        token1.append(string)

5.儲存完成文字前處理的分詞為token1

In [9]:
f= open("token1.txt","w+")
for i in range( len(token1) ):
    f.write(token1[i])
    if i != len(token1)-1:
        f.write(',')
f.close()

6.觀察分詞詞頻、TF-IDF

In [10]:
data_processing2 = ''
for i in range( len(token1) ):
    data_processing2 = data_processing2 + str( token1[i] )
jieba.load_userdict ('userdict.txt')
tfidf1 = jieba.analyse.extract_tags(data_processing2, topK=len(token1), withWeight=True)
tfidf1 = dict(tfidf1)
word1 = Counter(token1)
word1 = dict(word1)

In [11]:
for k, v in tfidf1.items():
    if k in word1:
        print(k, word1[k], round(v,4), end='\t\t\t')

野餐墊 11 0.2055			比較 9 0.1681			攤位 8 0.1494			衣架 8 0.1363			價錢 7 0.1308			標價 7 0.1308			衣服 11 0.1046			桌子 9 0.0993			過去 5 0.0934			桌布 5 0.0818			吊衣桿 4 0.0747			和室桌 4 0.0747			可愛 4 0.0747			時間 4 0.0747			討論 4 0.0747			擺攤 4 0.0747			六晚 1 0.0747			924 4 0.0747			925 4 0.0747			二手 5 0.0745			市集 4 0.067			行李箱 4 0.0663			地板 5 0.0637			一起 8 0.0609			公館 3 0.056			時候 3 0.056			啊啊 3 0.056			右邊 3 0.056			101 3 0.056			紙袋 3 0.056			拿出來 3 0.056			多少 6 0.0495			西瓜汁 2 0.0434			方便 4 0.0394			買下來 2 0.0374			衣桿 2 0.0374			飾品 2 0.0374			剛剛 2 0.0374			專業 2 0.0374			那麼 2 0.0374			好可愛 2 0.0374			大耳 2 0.0374			成為 2 0.0374			扭蛋 2 0.0374			獵奇 2 0.0374			很醜 2 0.0374			圍巾 2 0.0374			報名 2 0.0374			小鄧 2 0.0374			範圍 2 0.0374			什麼 2 0.0374			搞錯 2 0.0374			以為 2 0.0374			問題 2 0.0374			進去 2 0.0374			中間 2 0.0374			會不會 2 0.0374			喜歡 2 0.0374			天氣 2 0.0374			要賣 2 0.0374			之後 2 0.0374			吊牌 2 0.0374			算同 2 0.0374			紙膠帶 2 0.0374			小燈 2 0.0374			問問 2 0.0374			歐虧 2 0.0374			到時候 2 0.0374			框框 2 0.0346			地方 5 0.0345			弟弟 3 0.0338	

7.取TF-IDF前60高的分詞（stopwords(1)文字雲中的詞），除去詞頻為2的詞，儲存為keyword1.txt

In [12]:
i=0
for k, v in tfidf1.items():
    if k in word1:
        i+=1
        print(k, word1[k], round(v,4), end='\t\t\t')
    if i == 60:
        break;

野餐墊 11 0.2055			比較 9 0.1681			攤位 8 0.1494			衣架 8 0.1363			價錢 7 0.1308			標價 7 0.1308			衣服 11 0.1046			桌子 9 0.0993			過去 5 0.0934			桌布 5 0.0818			吊衣桿 4 0.0747			和室桌 4 0.0747			可愛 4 0.0747			時間 4 0.0747			討論 4 0.0747			擺攤 4 0.0747			六晚 1 0.0747			924 4 0.0747			925 4 0.0747			二手 5 0.0745			市集 4 0.067			行李箱 4 0.0663			地板 5 0.0637			一起 8 0.0609			公館 3 0.056			時候 3 0.056			啊啊 3 0.056			右邊 3 0.056			101 3 0.056			紙袋 3 0.056			拿出來 3 0.056			多少 6 0.0495			西瓜汁 2 0.0434			方便 4 0.0394			買下來 2 0.0374			衣桿 2 0.0374			飾品 2 0.0374			剛剛 2 0.0374			專業 2 0.0374			那麼 2 0.0374			好可愛 2 0.0374			大耳 2 0.0374			成為 2 0.0374			扭蛋 2 0.0374			獵奇 2 0.0374			很醜 2 0.0374			圍巾 2 0.0374			報名 2 0.0374			小鄧 2 0.0374			範圍 2 0.0374			什麼 2 0.0374			搞錯 2 0.0374			以為 2 0.0374			問題 2 0.0374			進去 2 0.0374			中間 2 0.0374			會不會 2 0.0374			喜歡 2 0.0374			天氣 2 0.0374			要賣 2 0.0374			

In [13]:
k_word1 = []
for k, v in tfidf1.items():
    if k in word1:
        i+=1
        if word1[k]>2:
            k_word1.append(k)
    if i == 60:
        break;
f= open("keyword1.txt","w+")
for i in range(len(k_word1)):
    f.write(k_word1[i])
    if i != len(k_word1)-1:
        f.write(',')
f.close()

#### TextProcessing--stop words(2.3)

In [14]:
jieba.load_userdict ('userdict.txt')
data_word2 = jieba.lcut( data_processing1 )
stopword2 = np.loadtxt( "stopword2-3.txt", dtype=str, delimiter=',')
i=1
while i!=0:
    i=0
    for string in data_word2:
        if string in stopword2:
            data_word2.remove(string)
            i += 1

In [15]:
for i in range(len(data_word2)):
    if( data_word2[i]=='24'): data_word2[i]='924'
    elif( data_word2[i]=='25'): data_word2[i]='925'
    elif( data_word2[i]=='9'): data_word2[i]=''
    elif( data_word2[i]=='10'): data_word2[i]='101'
    elif( data_word2[i]=='1'): data_word2[i]=''

In [16]:
cn = ['˙','ˊ','ˇ','ˋ']
token2=[]
for string in data_word2:
    if string.isalpha() and not(string.islower()) and (string not in cn) or string.isdigit() :
        token2.append(string)

In [17]:
f= open("token2.txt","w+")
for i in range( len(token2) ):
    f.write(token2[i])
    if i != len(token2)-1:
        f.write(',')
f.close()

In [18]:
data_processing2 = ''
for i in range( len(token2) ):
    data_processing2 = data_processing2 + str( token2[i] )
jieba.load_userdict ('userdict.txt')
tfidf2 = jieba.analyse.extract_tags(data_processing2, topK=len(data_word2), withWeight=True)
tfidf2 = dict(tfidf2)
word2 = Counter(token2)
word2 = dict(word2)

In [19]:
for k, v in tfidf2.items():
    if k in word2:
        print(k, word2[k], round(v,4), end='\t\t\t')

野餐墊 11 0.19			衣架 8 0.1261			攤位 8 0.1209			價錢 7 0.1209			標價 7 0.1209			這麼 6 0.1037			衣服 11 0.0967			過去 5 0.0864			桌子 9 0.0816			桌布 5 0.0756			吊衣桿 4 0.0691			和室桌 4 0.0691			可愛 4 0.0691			討論 4 0.0691			擺攤 4 0.0691			狀況 4 0.0691			六晚 1 0.0691			924 4 0.0691			925 4 0.0691			二手 5 0.0689			市集 4 0.062			行李箱 4 0.0613			不用 7 0.0608			地板 5 0.0589			如果 10 0.0573			大家 8 0.0563			那邊 3 0.0518			沒錯 3 0.0518			好了 3 0.0518			公館 3 0.0518			啊啊 3 0.0518			雖然 3 0.0518			右邊 3 0.0518			101 3 0.0518			確定 3 0.0518			紙袋 3 0.0518			拿出來 3 0.0518			還要 3 0.0518			200 3 0.0518			有沒有 3 0.0518			他們 3 0.0518			好笑 4 0.0486			多少 6 0.0457			西瓜汁 2 0.0402			好棒 2 0.0382			方便 4 0.0364			買下來 2 0.0346			衣桿 2 0.0346			飾品 2 0.0346			之類 2 0.0346			剛剛 2 0.0346			專業 2 0.0346			那麼 2 0.0346			好可愛 2 0.0346			大耳 2 0.0346			成為 2 0.0346			扭蛋 2 0.0346			獵奇 2 0.0346			很醜 2 0.0346			怎樣 2 0.0346			圍巾 2 0.0346			報名 2 0.0346			小鄧 2 0.0346			範圍 2 0.0346			搞錯 2 0.0346			以為 2 0.0346			問題 2 0.0346			進去 2 0.0346			中間 2 0.0346			會不會 2 0.0346			喜歡 2 0

．取TF-IDF前60高的分詞（stopwords(2)文字雲中的詞），除去詞頻為2的詞，其中保留TEIDF>0.052的分詞，儲存為keyword2.txt

In [20]:
i=0
for k, v in tfidf2.items():
    if k in word2:
        i+=1
        print(k, word2[k], round(v,4), end='\t\t\t')
    if i == 60:
        break;

野餐墊 11 0.19			衣架 8 0.1261			攤位 8 0.1209			價錢 7 0.1209			標價 7 0.1209			這麼 6 0.1037			衣服 11 0.0967			過去 5 0.0864			桌子 9 0.0816			桌布 5 0.0756			吊衣桿 4 0.0691			和室桌 4 0.0691			可愛 4 0.0691			討論 4 0.0691			擺攤 4 0.0691			狀況 4 0.0691			六晚 1 0.0691			924 4 0.0691			925 4 0.0691			二手 5 0.0689			市集 4 0.062			行李箱 4 0.0613			不用 7 0.0608			地板 5 0.0589			如果 10 0.0573			大家 8 0.0563			那邊 3 0.0518			沒錯 3 0.0518			好了 3 0.0518			公館 3 0.0518			啊啊 3 0.0518			雖然 3 0.0518			右邊 3 0.0518			101 3 0.0518			確定 3 0.0518			紙袋 3 0.0518			拿出來 3 0.0518			還要 3 0.0518			200 3 0.0518			有沒有 3 0.0518			他們 3 0.0518			好笑 4 0.0486			多少 6 0.0457			西瓜汁 2 0.0402			好棒 2 0.0382			方便 4 0.0364			買下來 2 0.0346			衣桿 2 0.0346			飾品 2 0.0346			之類 2 0.0346			剛剛 2 0.0346			專業 2 0.0346			那麼 2 0.0346			好可愛 2 0.0346			大耳 2 0.0346			成為 2 0.0346			扭蛋 2 0.0346			獵奇 2 0.0346			很醜 2 0.0346			怎樣 2 0.0346			

In [21]:
k_word2 = []
for k, v in tfidf2.items():
    if k in word2 and word2[k]>2 or v>=0.052:
        k_word2.append(k)
f= open("keyword2.txt","w+")
for i in range(len(k_word2)):
    f.write(k_word2[i])
    if i != len(k_word2)-1:
        f.write(',')
f.close()

### Segmenting sentences -> Word2Vec
#### 沒設置停用詞

In [22]:
data_processing1 = ''
for string in data_text:
    data_processing1 = data_processing1 + string + '\n'
jieba.load_userdict ('userdict.txt')
data_sentence = jieba.lcut( data_processing1 )
for i in range(len(data_sentence)):
    if( data_sentence[i]=='24'): data_sentence[i]='924'
    elif( data_sentence[i]=='25'): data_sentence[i]='925'
    elif( data_sentence[i]=='9'): data_sentence[i]=''
    elif( data_sentence[i]=='10'): data_sentence[i]='101'
    elif( data_sentence[i]=='1'): data_sentence[i]=''
sentence_words = []
l = []
for string in data_sentence:
    if string in '\n':
        sentence_words.append(l)
        del l
        l = []
        continue;
    l.append(string) 

In [23]:
f= open("sentence_token0.txt","w+")
for i in range( len(sentence_words) ):
    for j in range( len(sentence_words[i]) ):
        string = sentence_words[i][j]
        f.write(string + ' ')
    f.write(string + '\n')
f.close()

#### 設置stop words(1)

In [24]:
data_processing1 = ''
data_sentence1 = []
for sentence in data_text:
    jieba.load_userdict ('userdict.txt')
    s = jieba.lcut( sentence )
    for i in range(len(s)):
        if( s[i]=='24'): s[i]='924'
        elif( s[i]=='25'): s[i]='925'
        elif( s[i]=='9'): s[i]=''
        elif( s[i]=='10'): s[i]='101'
        elif( s[i]=='1'): s[i]=''
    l = []
    for string in s:
        if string in token1:
            l.append(string)
    if len(l)!=0:
        data_sentence1.append(l)
data_sentence1

[['惹來'],
 ['姚'],
 ['最多', '衣服', '猜', '姚'],
 ['讚讚'],
 ['好好', '過濾'],
 ['請貼', '上衣', '竿子'],
 ['三',
  '一起',
  '擺',
  '要租',
  '兩',
  '攤位',
  '一',
  '攤位',
  '放',
  '桌子',
  '一',
  '攤位',
  '放',
  '吊衣桿',
  '行李箱',
  '野餐墊'],
 ['規定', '不能', '放', '行李箱', '野餐墊'],
 ['野餐墊', '方便'],
 ['家裡', '桌子'],
 ['逛', '二手拍', '野餐墊', '加', '行李箱'],
 ['擠', '蹲下來', '翻', '衣服'],
 ['李', '佳芬', '請', '分享', '今天', '心得'],
 ['真的', '擠'],
 ['路過', '翻'],
 ['偏向', '一', '吊衣桿', '花', '錢', '買下來'],
 ['知道', '張譽', '借', '和室桌'],
 ['衣桿'],
 ['譽', '桌子', '張醬'],
 ['長'],
 ['放', '野餐墊'],
 ['等下', '畫', '平面圖'],
 ['大便'],
 ['放', '飾品'],
 ['剛剛', '喝', '西瓜汁'],
 ['大便'],
 ['非常', '專業'],
 ['桌布', '那麼'],
 ['西瓜汁', '好喝'],
 ['庫洛米', '好可愛'],
 ['免費', '野餐墊'],
 ['大耳', '狗'],
 ['立志', '成為', '公館', '市集', '年輕', '時尚', '攤位'],
 ['野餐墊'],
 ['啤酒', '打卡', '送'],
 ['那天', '扭', '一', '大耳', '狗', '蠟筆', '新', '聯名', '扭蛋'],
 ['長得', '獵奇'],
 ['很醜', '好用'],
 ['攝影棚', '翻出來'],
 ['虧本', '唄'],
 ['到底', '公司', '偷', '多少'],
 ['而且', '很醜'],
 ['直接', '變成', '專業'],
 ['墊子', '長'],
 ['真假'],
 ['拍'],
 ['一', '小角'],
 ['比較', '可愛'],
 ['

In [25]:
f= open("sentence_token1.txt","w+")
for i in range( len(data_sentence1) ):
    for j in range( len(data_sentence1[i]) ):
        string = data_sentence1[i][j]
        f.write(string + ' ')
    f.write(string + '\n')
f.close()

#### 設置stop words(2.3)

In [26]:
data_processing2 = ''
data_sentence2 = []
for sentence in data_text:
    jieba.load_userdict ('userdict.txt')
    s = jieba.lcut( sentence )
    for i in range(len(s)):
        if( s[i]=='24'): s[i]='924'
        elif( s[i]=='25'): s[i]='925'
        elif( s[i]=='9'): s[i]=''
        elif( s[i]=='10'): s[i]='101'
        elif( s[i]=='1'): s[i]=''
    l = []
    for string in s:
        if string in token2:
            l.append(string)
    if len(l)!=0:
        data_sentence2.append(l)
data_sentence2

[['惹來', '惹'],
 ['姚'],
 ['最多', '衣服', '猜', '姚'],
 ['讚讚'],
 ['好好', '過濾'],
 ['請貼', '上衣', '竿子'],
 ['三', '擺', '要租', '攤位', '攤位', '放', '桌子', '另', '攤位', '放', '吊衣桿', '行李箱', '野餐墊'],
 ['規定', '不能', '放', '行李箱', '和', '野餐墊'],
 ['野餐墊', '最', '方便'],
 ['如果', '家裡', '小', '桌子', '呢'],
 ['逛', '二手拍', '野餐墊', '加', '行李箱'],
 ['那邊', '擠', '蹲下來', '翻', '衣服'],
 ['李', '佳芬', '請', '分享', '心得'],
 ['沒錯', '擠'],
 ['路過', '翻'],
 ['偏向', '吊衣桿', '如果', '大家', '花', '錢', '買下來', '用', '蛤'],
 ['張譽', '借', '和室桌'],
 ['衣桿'],
 ['譽', '桌子', '張醬'],
 ['長'],
 ['放', '野餐墊', '那邊'],
 ['等下', '畫', '平面圖', '好了'],
 ['大便'],
 ['放', '飾品', '之類'],
 ['剛剛', '喝', '西瓜汁'],
 ['大便'],
 ['非常', '專業'],
 ['大家', '桌布', '不用', '那麼'],
 ['西瓜汁', '好喝'],
 ['庫洛米', '好可愛'],
 ['ㄟ', 'ㄟ', 'ㄟ'],
 ['免費', '野餐墊'],
 ['大耳', '狗'],
 ['立志', '成為', '公館', '市集', '最', '年輕', '時尚', '攤位'],
 ['野餐墊'],
 ['啤酒', '打卡', '送'],
 ['那天', '扭', '大耳', '狗', '蠟筆', '小', '新', '聯名', '扭蛋'],
 ['長得', '獵奇'],
 ['很醜', '好用'],
 ['攝影棚', '翻出來'],
 ['虧本', '唄'],
 ['到底', '從', '公司', '偷', '多少'],
 ['而且', '很醜'],
 ['直接', '變成', '專業'],
 ['墊子', '

In [27]:
f= open("sentence_token2.txt","w+")
for i in range( len(data_sentence2) ):
    for j in range( len(data_sentence2[i]) ):
        string = data_sentence2[i][j]
        f.write(string + ' ')
    f.write(string + '\n')
f.close()